In [1]:
import what.utils.logger as log

logger = log.get_logger(__name__)

In [2]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
pretrained_ssd = "models/mobilenet-v2-ssd-lite-mp-0_686.pth"
checkpoint_folder = "models/"

# Load Training Dataset

In [4]:
batch_size = 32
num_workers = 0

In [5]:
import fiftyone as fo
import fiftyone.zoo as foz
from torch.utils.data import DataLoader

from what.models.detection.ssd.datasets.fiftyone import FiftyOneDataset

In [6]:
from what.models.detection.ssd.ssd.preprocessing import TrainAugmentation, TestTransform
from what.models.detection.ssd.ssd.config import mobilenetv1_ssd_config
from what.models.detection.ssd.ssd.ssd import MatchPrior

In [7]:
# Load MobileNetSSD configuration
config              = mobilenetv1_ssd_config
train_transform     = TrainAugmentation(config.image_size, config.image_mean, config.image_std)
target_transform    = MatchPrior(config.priors, config.center_variance, config.size_variance, 0.5)
test_transform      = TestTransform(config.image_size, config.image_mean, config.image_std)

In [8]:
# Load the data from fiftyone
voc_2012_train = foz.load_zoo_dataset("voc-2012", split="train")

Split 'train' already downloaded
Loading existing dataset 'voc-2012-train'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [9]:
# Visualize the dataset
session = fo.launch_app(voc_2012_train)

Connected to FiftyOne on port 5151 at 127.0.0.1.
If you are not connecting to a remote session, you may need to start a new session and specify a port


In [10]:
# Convert to pytorch dataloader
train_dataset = FiftyOneDataset(voc_2012_train, 
                                foz.load_zoo_dataset_info("voc-2012").classes, 
                                transform=train_transform,
                                target_transform=target_transform)

# Load Training Dataset from Local Disk
# train_dataset = VOCDataset("examples/VOC2012", transform=train_transform,
#                              target_transform=target_transform)

train_loader  = DataLoader(train_dataset, batch_size,
                          num_workers=num_workers,
                          shuffle=True)

logger.info("Train dataset size: {}".format(len(train_dataset)))

[INFO] 2022-01-14 16:43:25,413 __main__: Train dataset size: 5717


# Load Validation Dataset

In [11]:
# Load Validation Dataset from FiftyOne (use voc-2007 train as validation here)
val_dataset  = FiftyOneDataset(foz.load_zoo_dataset("voc-2007", split="train"), 
                                        foz.load_zoo_dataset_info("voc-2007").classes,
                                        transform=test_transform,
                                        target_transform=target_transform)

# Load Validation Dataset from Local Disk
# val_dataset = VOCDataset("VOC2007/", transform=test_transform,
#                              target_transform=target_transform, is_test=True)

val_loader = DataLoader(val_dataset, batch_size,
                        num_workers=num_workers,
                        shuffle=False)

logger.info("validation dataset size: {}".format(len(val_dataset)))

Split 'train' already downloaded
Loading existing dataset 'voc-2007-train'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


[INFO] 2022-01-14 16:43:25,514 __main__: validation dataset size: 2501


# Train Mobilenet V2 SSD Lite

In [12]:
from what.models.detection.ssd.mobilenet_v2_ssd_lite import MobileNetV2SSDLite

In [13]:
# Create SSD network and load pretrained base net.
model = MobileNetV2SSDLite(is_test=False)

model.train(train_loader, val_loader, device=device, num_epochs=1, debug_steps=10, validation_epochs=1,
           pretrained_ssd = pretrained_ssd, checkpoint_folder = checkpoint_folder)

[INFO] 2022-01-14 16:43:25,687 what.models.detection.ssd.mobilenet_v2_ssd_lite: Init from pretrained ssd models/mobilenet-v2-ssd-lite-mp-0_686.pth
[INFO] 2022-01-14 16:43:25,778 what.models.detection.ssd.mobilenet_v2_ssd_lite: Start training using CosineAnnealingLR scheduler.
[INFO] 2022-01-14 16:45:32,873 what.models.detection.ssd.mobilenet_v2_ssd_lite: Epoch: 0, Step: 10, Average Loss: 22.4287, Average Regression Loss 4.4760, Average Classification Loss: 17.9527
[INFO] 2022-01-14 16:47:33,999 what.models.detection.ssd.mobilenet_v2_ssd_lite: Epoch: 0, Step: 20, Average Loss: 16.8115, Average Regression Loss 3.1899, Average Classification Loss: 13.6216
[INFO] 2022-01-14 16:49:14,377 what.models.detection.ssd.mobilenet_v2_ssd_lite: Epoch: 0, Step: 30, Average Loss: 14.7194, Average Regression Loss 2.7147, Average Classification Loss: 12.0047
[INFO] 2022-01-14 16:51:05,994 what.models.detection.ssd.mobilenet_v2_ssd_lite: Epoch: 0, Step: 40, Average Loss: 13.0328, Average Regression Loss 